**Esercizi Neo4j 2**

In [ ]:
!pip install neo4j-driver

     |████████████████████████████████| 89 kB 3.8 MB/s 
  Created wheel for neo4j-driver: filename=neo4j_driver-4.4.0-py3-none-any.whl size=114976 sha256=1d20b742f66c6bb93592b2c1b4c3daa69649de8ac6fee72d5c097dbf6cf4c50b
  Stored in directory: /root/.cache/pip/wheels/5e/89/23/48d22f30bfad0866aac1078fb2e24db1dd3750c46198557ffe
Successfully built neo4j-driver


In [ ]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd
pd.options.display.max_rows = 999

In [ ]:
driver = GraphDatabase.driver("bolt://3.235.133.192:7687", auth=basic_auth("neo4j", "jurisdictions-walks-symbol"))

Prendere in considerazione su Neo4j sandbox il database “Women's World Cup 2019” ed utilizzare il comando
```
call db.schema.visualization()
```
per avere un quadro dei nodi e degli archi che lo compongono. Il database contiene le informazioni della Women's World Cup dal 1999 al 2019.

Realizzare poi un notebook Python per estrarre le seguenti informazioni:


In [ ]:
# 1. i nomi delle squadre che hanno partecipano al torneo nel 2015
cypher_query = '''
MATCH (t:Team)-[p:PARTICIPATED_IN]-(T:Tournament {year: 2015})
RETURN t.name AS nome
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df1 = pd.DataFrame(result)
df1

,nome
0,Korea Republic
1,Germany
2,Nigeria
3,CÃ´te d'Ivoire
4,Norway
5,Thailand
6,Canada
7,Ecuador
8,China PR
9,Japan


In [ ]:
# 2. i nomi delle squadre che hanno partecipato al torneo nel 2015 (farlo in un altro modo)
cypher_query = '''
MATCH (t:Team)-[n:NAMED]-(s:Squad)-[f:FOR]-(T:Tournament)
WHERE s.id CONTAINS 'in 2015'
RETURN t.name AS nome
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df2 = pd.DataFrame(result)
df2

,nome
0,Australia
1,Brazil
2,Cameroon
3,Canada
4,China PR
5,Colombia
6,Costa Rica
7,CÃ´te d'Ivoire
8,Ecuador
9,England


In [ ]:
# 3. il numero di partecipazioni di ogni nazione
cypher_query = '''
MATCH (t:Team)-[p:PARTICIPATED_IN]-(T:Tournament)
RETURN t.name AS nome, COUNT(p) AS totale
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df3 = pd.DataFrame(result)
df3

,nome,totale
0,Equatorial Guinea,1
1,Nigeria,8
2,Australia,7
3,Japan,8
4,Korea DPR,4
5,England,5
6,France,4
7,Germany,8
8,Norway,8
9,Sweden,8


In [ ]:
# 4. gli allenatori dell'Italia
cypher_query = '''
MATCH (t:Team {name: 'Italy'})-[n:NAMED]-(s:Squad)-[c:COACH_FOR]-(p:Person)
RETURN p.name AS allenatori
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df4 = pd.DataFrame(result)
df4

,allenatori
0,Facchin Carlo
1,Milena Bertolini
2,Guenza Sergio


In [ ]:
# 5. le giocatrici dell'Italia che hanno segnato e in quale partita hanno segnato (data)
cypher_query = '''
MATCH (m:Match)-[s:SCORED_GOAL]-(p:Person)-[r:REPRESENTS]-(t:Team {name: 'Italy'})
RETURN p.name AS nome, m.date AS data
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df5 = pd.DataFrame(result)
df5

,nome,data
0,Raffaella Saimaso,1991-11-24
1,Paola Zanni,1999-06-27
2,Patrizia Panico,1999-06-27
3,Patrizia Panico,1999-06-20
4,Barbara Bonansea,2019-06-09
5,Barbara Bonansea,2019-06-09
6,Rita Guarino,1991-11-24
7,Aurora Galli,2019-06-14
8,Aurora Galli,2019-06-14
9,Aurora Galli,2019-06-25


In [ ]:
# 6. il numero di gol segnato dalle giocatrici italiane in totale
cypher_query = '''
MATCH (m:Match)-[s:SCORED_GOAL]-(p:Person)-[r:REPRESENTS]-(t:Team {name: 'Italy'})
RETURN COUNT(p) as totale
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df6 = pd.DataFrame(result)
df6

,totale
0,20


In [ ]:
# 7. le giocatrici che hanno partecipato alla partita del 9 giugno 2019
cypher_query = '''
MATCH (a:Person)-[p:PLAYED_IN]-(m:Match {date: date('2019-06-09')})
RETURN a.name AS nome
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df7 = pd.DataFrame(result)
df7

,nome
0,Hayley Raso
1,Lydia Williams
2,Sara Gama
3,Elise Kellond-Knight
4,Valentina Bergamaschi
5,Alanna Kennedy
6,Caitlin Foord
7,Elisa Bartoli
8,Samantha Kerr
9,Ellie Carpenter


In [ ]:
# 8. le giocatrici che hanno segnato nella partita del 9 giugno 2019
cypher_query = '''
MATCH (a:Person)-[p:SCORED_GOAL]-(m:Match {date: date('2019-06-09')})
RETURN a.name AS nome
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df8 = pd.DataFrame(result)
df8

,nome
0,Samantha Kerr
1,Barbara Bonansea
2,Barbara Bonansea
3,Nikita Parris
4,Claire Emslie
5,Ellen White
6,Cristiane
7,Cristiane
8,Cristiane


In [ ]:
# 9. le giocatrici che non hanno segnato nella partita del 9 giugno 2019
# google it: https://neo4j.com/developer/cypher/filtering-query-results/#filter-patterns
cypher_query = '''
MATCH (a:Person)-[p:PLAYED_IN]-(m:Match {date: date('2019-06-09')})
WHERE NOT exists((a)-[:SCORED_GOAL]-(m))
RETURN a.name AS nome
'''

with driver.session() as graphDB_Session:
    result = graphDB_Session.run(cypher_query).data()
df9 = pd.DataFrame(result)
df9

,nome
0,Hayley Raso
1,Lydia Williams
2,Sara Gama
3,Elise Kellond-Knight
4,Valentina Bergamaschi
5,Alanna Kennedy
6,Caitlin Foord
7,Elisa Bartoli
8,Ellie Carpenter
9,Valentina Giacinti
